<a href="https://colab.research.google.com/github/HadiAnik/pyhton-code/blob/main/Using_Databases_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***2 Our First Database----- Using Databases with Python***

In [39]:
import sqlite3

In [40]:
# Create a database in RAM
db = sqlite3.connect(':memory:')

# Get a cursor object
cursor = db.cursor()
cursor.execute('''
    CREATE TABLE Ages ( 
  name VARCHAR(128), 
  age INTEGER
)''')


cursor.execute('''DELETE FROM Ages''')

In [41]:
# Insert users
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Bertie', 23)''')
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Tanika', 19)''')
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Brogan', 21)''')
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Meryl', 39)''')

#Select user
cursor.execute('''SELECT hex(name || age) AS X FROM Ages ORDER BY X''')

#retrieve the first row
user1 = cursor.fetchone()
#Print the first column retrieved(user's name)
print("The first row in the resulting record set : "+user1[0])

#Commit changes into database
db.commit()
#Close database
db.close()

The first row in the resulting record set : 4265727469653233


## **Complete first Assignment** 😎😍😍

## **-----------------------------------------------------------------------------------------------------------------**

## **2nd Assignment  Counting Email in a Database**

In [42]:
import sqlite3

conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('''
DROP TABLE IF EXISTS Counts''')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'mbox.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From: ') : continue
    pieces = line.split()
    email = pieces[1]
    parts = email.split('@')
    org = parts[-1]
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org, ))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count) 
                VALUES ( ?, 1 )''', ( org, ) )
    else :
        cur.execute('UPDATE Counts SET count=count+1 WHERE org = ?',
            (org, ))
    conn.commit()

sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

print
print("Counts:")
for row in cur.execute(sqlstr) :
    print (str(row[0]), row[1])

cur.close()

Enter file name: mbox.txt
Counts:
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


##  **2nd Part is Completed** 😍😎

## **-----------------------------------------------------------------------------------------------------------------**

## ***3rd week Examination***

In [51]:
import sqlite3
import xml.etree.ElementTree as ET

In [52]:
#Function that we'll use to find the content of a specific field.
def find_field(track, wanted_field):
    """This function gets two parameters: track, a dictionary containing all
    the XML tags of a certain song, and wanted_field, a string representing the
    title of the tag we want to obtain.
    It works by finding a key tag with the text {wanted_field}, and
    returning the content of the following tag. If wanted_field doesn't
    match any tag, it returns a False"""

    #Variable we'll use to indicate when we've found wanted_field
    found = False

    for tag in track:
        if not found:
            #Looking for the wanted field
            if(tag.tag == "key" and tag.text == wanted_field):
                found = True
        else:
            #After founding it, we return the content of the following
            #tag (the one with its value)
            return tag.text

    return False



#PART 1: PREPARING THE DATABASE
#Connecting to the file in which we want to store our db
conn = sqlite3.connect('tracks.sqlite')
cur = conn.cursor()

#Getting sure it is empty
#We can use "executescript" to execute several statements at the same time
cur.executescript("""
    DROP TABLE IF EXISTS Artist;
   
    DROP TABLE IF EXISTS Album; 
    DROP TABLE IF EXISTS Genre;
    DROP TABLE IF EXISTS Track
    """)

#Creating it
cur.executescript(''' CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);
CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


#PART 2: INSERTING THE DATA
#Getting the data and parsing it
data_source = open("Library.xml")
data = data_source.read()
xml_data = ET.fromstring(data)

#Obtaining every tag with track data
tracks_data = xml_data.findall("dict/dict/dict")

#Getting the values of the fields we'll insert
for track in tracks_data:
    title = find_field(track, "Name")
    artist = find_field(track, "Artist")
    genre = find_field(track, "Genre")
    album = find_field(track, "Album")
    length = find_field(track, "Total Time")
    count = find_field(track, "Play Count")
    rating = find_field(track, "Rating")

    #Artist
    if (artist): #If it's a filled string, != False
        #If the value hasn't been introduced yet and exists, we'll insert it
        artist_statement = """INSERT INTO Artist(name) SELECT ? WHERE NOT EXISTS 
            (SELECT * FROM Artist WHERE name = ?)"""
        SQLparams = (artist, artist) #Params needed for completing the statement
        cur.execute(artist_statement, SQLparams)

    #Genre
    if (genre): #If it's a filled string, != False
        #If the value hasn't been introduced yet and exists, we'll insert it
        genre_statement = """INSERT INTO Genre(name) SELECT ? WHERE NOT EXISTS 
            (SELECT * FROM Genre WHERE name = ?)"""
        SQLparams = (genre, genre)
        cur.execute(genre_statement, SQLparams)

    #Album
    if (album): #If it's a filled string, != False
        #First of all, we'll get the artist id
        artistID_statement = "SELECT id from Artist WHERE name = ?"
        cur.execute(artistID_statement, (artist, ))
        #.fetchone() returns a one-element tuple, and we want its content
        artist_id = cur.fetchone()[0]


        #Now we're going to insert the data
        album_statement = """INSERT INTO Album(title, artist_id) 
            SELECT ?, ? WHERE NOT EXISTS (SELECT * FROM Album WHERE title = ?)"""
        SQLparams = (album, artist_id, album)
        cur.execute(album_statement, SQLparams)

    #Track
    if (title): #If it's a filled string, != False
        #Obtaining genre_id
        genreID_statement = "SELECT id from Genre WHERE name = ?"
        cur.execute(genreID_statement, (genre, ))
        try:
            genre_id = cur.fetchone()[0]
        except TypeError:
            genre_id = 0
        #Obtaining album_id
        albumID_statement = "SELECT id from Album WHERE title = ?"
        cur.execute(albumID_statement, (album, ))
        try:
            album_id = cur.fetchone()[0]
        except TypeError:
            album_id = 0

        #Inserting data
        track_statement = """INSERT INTO Track(title, album_id, genre_id, len,
            rating, count) SELECT ?, ?, ?, ?, ?, ?
                WHERE NOT EXISTS (SELECT * FROM Track WHERE title = ?)"""
        SQLparams = (title, album_id, genre_id, length, rating, count, title)
        cur.execute(track_statement, SQLparams)


conn.commit()
cur.close()

# ***Assignment 3 is completed *** 😍


## **-----------------------------------------------------------------------------------------------------------------**

## ***4rd week Examination***

In [78]:
# import json
# import sqlite3

# #PART 1: Creating the database
# dbname = "roster.sqlite"
# conn = sqlite3.connect(dbname)
# cur = conn.cursor()

In [97]:
import json
import sqlite3

#PART 1: Creating the database
dbname = "roster.sqlite"
conn = sqlite3.connect(dbname)
cur = conn.cursor()

cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;

CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);

CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);

CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

#Note: if we don't add UNIQUE after "User.name" and "Course.title", 
#the IGNORE statement won't work and therefore we'll have duplicates


#PART 2: DESERIALIZING THE data
#The JSON data we're going to process is stored in an array form, with each
#item being also an array of three elements: one corresponding to the username 
#one corresponding to the course name, and one indicating if the user is role
#None of them has any field title. 

fname = "roster_data.json"
if len(fname) < 1:
    fname = 'roster_data_sample.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0]
    title = entry[1]
    role = entry[2]

    # print(entry)

    # print(name ,'|', title ,"|", role)

    cur.execute('''INSERT OR IGNORE INTO User (name) 
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        ( user_id, course_id, role ) )
    
    conn.commit()

#PART 4: Testing and obtaining the results
test_statement = """
SELECT 'XYZZY' || hex(User.name || Course.title || Member.role ) AS X FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY X"""
cur.execute(test_statement)
result = cur.fetchone()
print("The first row in the resulting record set: ", result)

#Closing the connection
cur.close()
conn.close()


The first row in the resulting record set:  ('XYZZY41616D616E6565736933363430',)


# ***Assignment 4 is completed *** 😍


## **-----------------------------------------------------------------------------------------------------------------**

## ***5rd week Examination***